In [1]:
import pandas as pd
import numpy as np
import googlemaps
import json
from itertools import combinations
import pickle

# Google Maps API
Tests on Google Maps API

In [18]:
# Get API key
f = open("../config.json")
API = json.load(f)
API_key = API["API_key"]

maps = googlemaps.Client(key = API_key)

In [144]:
# Test distance
start_point = "Villafranca di Verona"
end_point = "Trento"

distance = maps.directions(start_point, end_point)

In [145]:
distance

[{'bounds': {'northeast': {'lat': 46.0775632, 'lng': 11.1265087},
   'southwest': {'lat': 45.355234, 'lng': 10.7798422}},
  'copyrights': 'Map data ©2023',
  'legs': [{'distance': {'text': '106 km', 'value': 105825},
    'duration': {'text': '1 hour 14 mins', 'value': 4418},
    'end_address': 'Trento, Autonomous Province of Trento, Italy',
    'end_location': {'lat': 46.0747235, 'lng': 11.1219304},
    'start_address': '37069 Villafranca di Verona, VR, Italy',
    'start_location': {'lat': 45.355234, 'lng': 10.8474199},
    'steps': [{'distance': {'text': '0.2 km', 'value': 244},
      'duration': {'text': '1 min', 'value': 37},
      'end_location': {'lat': 45.3563585, 'lng': 10.8447541},
      'html_instructions': 'Head <b>northwest</b> on <b>Via Gaetano Bellotti</b>',
      'polyline': {'points': 'emysGkseaAA@ANGPWz@IZ_BjFo@rBIZY\\'},
      'start_location': {'lat': 45.355234, 'lng': 10.8474199},
      'travel_mode': 'DRIVING'},
     {'distance': {'text': '0.2 km', 'value': 167},
 

In [16]:
distance[0].keys()

dict_keys(['bounds', 'copyrights', 'legs', 'overview_polyline', 'summary', 'warnings', 'waypoint_order'])

In [21]:
distance[0]['warnings']

[]

# Test with landmarks

In [19]:
landmarks_10 = ["Paris, France",
            "Istanbul, Turkey",
            "Rome, Italy",
            "Amsterdam, Netherlands",
            "Barcelona, Spain",
            "Prague, Czech Republic",
            "Vienna, Austria",
            "Milan, Italy",
            "Athens, Greece",
            "Berlin, Germany"]
landmarks_50 = [
            "Paris, France",
            "London, United Kingdom",
            "Rome, Italy",
            "Istanbul, Turkey",
            "Barcelona, Spain",
            "Prague, Czech Republic",
            "Amsterdam, Netherlands",
            "Vienna, Austria",
            "Berlin, Germany",
            "Madrid, Spain",
            "Florence, Italy",
            "Dublin, Ireland",
            "Lisbon, Portugal",
            "Budapest, Hungary",
            "Athens, Greece",
            "Edinburgh, Scotland",
            "Venice, Italy",
            "Munich, Germany",
            "Stockholm, Sweden",
            "Copenhagen, Denmark",
            "Zurich, Switzerland",
            "Dubrovnik, Croatia",
            "Brussels, Belgium",
            "Warsaw, Poland",
            "Krakow, Poland",
            "Oslo, Norway",
            "Porto, Portugal",
            "Helsinki, Finland",
            "Nice, France",
            "Seville, Spain",
            "Reykjavik, Iceland",
            "Lyon, France",
            "Geneva, Switzerland",
            "Marseille, France",
            "Palma de Mallorca, Spain",
            "Riga, Latvia",
            "Tallinn, Estonia",
            "Bucharest, Romania",
            "Sofia, Bulgaria",
            "Naples, Italy",
            "Valencia, Spain",
            "Bergen, Norway",
            "Ljubljana, Slovenia",
            "Santorini, Greece",
            "Salzburg, Austria",
            "Gdansk, Poland",
            "Antwerp, Belgium",
            "Split, Croatia",
            "Vilnius, Lithuania",
            "Tbilisi, Georgia"
            ]
landmarks_test = [
"Fortezza, Italy",
"Lubiana, Slovenia",
"Zagabria, Croatia",
"Budapest, Hungary",
"Cracovia, Poland",
"Praga, Czech Republic",
"Passau, Germany",
"Bolzano, Italy"
]

In [20]:
landmarks_distances = {}
landmarks_durations = {}

for (city_1, city_2) in combinations(landmarks_test, 2):
    try:
        travel_data = maps.distance_matrix(origins=[city_1], 
                                            destinations=[city_2],
                                            mode='driving',
                                            language='English',
                                            units='metric')
        
        distance = travel_data["rows"][0]["elements"][0]["distance"]["value"]

        duration = travel_data["rows"][0]["elements"][0]["duration"]["value"]

        landmarks_distances[frozenset([city_1, city_2])] = distance
        landmarks_durations[frozenset([city_1, city_2])] = duration
    except Exception as e:
        print("Error with finding the route between %s and %s." % (city_1, city_2))

In [15]:
# Save travel data to files
with open('landmarks_distances.pickle', 'wb') as f:
    pickle.dump(landmarks_distances, f)
with open('landmarks_durations.pickle', 'wb') as f:
    pickle.dump(landmarks_durations, f)


In [3]:
# Load travel data from files
with open('landmarks50_distances.pickle', 'rb') as f:
    landmarks_distances = pickle.load(f)
with open('landmarks50_durations.pickle', 'rb') as f:
    landmarks_durations = pickle.load(f)

In [4]:
len(landmarks_distances)

1225

In [18]:
landmarks_distances

{frozenset({'Fortezza, Italy', 'Lubiana, Slovenia'}): 309103,
 frozenset({'Fortezza, Italy', 'Zagabria, Croatia'}): 448663,
 frozenset({'Budapest, Hungary', 'Fortezza, Italy'}): 792324,
 frozenset({'Cracovia, Poland', 'Fortezza, Italy'}): 1007982,
 frozenset({'Fortezza, Italy', 'Praga, Czech Republic'}): 615030,
 frozenset({'Fortezza, Italy', 'Passau, Germany'}): 344464,
 frozenset({'Bolzano, Italy', 'Fortezza, Italy'}): 50654,
 frozenset({'Lubiana, Slovenia', 'Zagabria, Croatia'}): 140442,
 frozenset({'Budapest, Hungary', 'Lubiana, Slovenia'}): 462260,
 frozenset({'Cracovia, Poland', 'Lubiana, Slovenia'}): 837331,
 frozenset({'Lubiana, Slovenia', 'Praga, Czech Republic'}): 706522,
 frozenset({'Lubiana, Slovenia', 'Passau, Germany'}): 399560,
 frozenset({'Bolzano, Italy', 'Lubiana, Slovenia'}): 423492,
 frozenset({'Budapest, Hungary', 'Zagabria, Croatia'}): 344669,
 frozenset({'Cracovia, Poland', 'Zagabria, Croatia'}): 825881,
 frozenset({'Praga, Czech Republic', 'Zagabria, Croatia'}):

# Genetic Algorithm

In [45]:
def generate_random_solution(landmarks):
    """
        Generates a random solution by shuffling the waypoints.

        Arguments
        ---------
        landmarks (list): a list containing all the destinations to be visited

        Returns
        -------
        tuple: tuple containing a shuffled version of the original landmarks list
    """
    new_solution = landmarks.copy()
    np.random.shuffle(new_solution)

    return tuple(new_solution)

def generate_random_population(population_size, landmarks):
    """
        Generates a list of random solutions.

        Arguments
        ---------
        population_size (int): number of solutions to be generated
        landmarks (list): a list containing all the destinations to be visited

        Returns
        -------
        list: tuple containing the random solutions
    """
    random_population = []
    for i in range(population_size):
        random_population.append(generate_random_solution(landmarks))
    return random_population

def evaluate_solution(solution, landmarks_distances, landmarks_durations, kind='distance'):
    """
        Evaluates the solution's fitness by summing the total distance/time needed to travel to each 
        destination in that order.

        Arguments
        ---------
        solution (tuple): tuple containing a solution, i.e. an ordered list of destinations
        kind (string, optional): it specifies if the algorithm considers the total distance or the total time
            of the travel.
            Default='distance'

        Returns
        -------
        float: sum of the total travel distance/time relative to the input solution
    """
    fitness = 0.

    for idx in list(range(len(solution)))[1:]:
        start_point = solution[idx-1]
        end_point = solution[idx]

        if kind == 'distance':
            fitness += landmarks_distances[frozenset([start_point, end_point])]
        elif kind == 'time':
            fitness += landmarks_durations[frozenset([start_point, end_point])]
        else:
            raise Exception("Not recognized kind. Allowed values are 'distance' and 'time'.")
    return fitness

def crossover_mutation(solution):
    """
        Mutates a solution by slicing a random subset of its genome and appending it to another position.

        Arguments
        ---------
        solution (tuple): tuple containing a solution, i.e. an ordered list of destinations

        Returns
        -------
        tuple: tuple containing a solution, i.e. an ordered list of destinations. Different from the input
    """
    # print(solution)
    # Take 2 random indices for slicing
    idx_1, idx_2 = np.random.choice(range(len(solution)+1),2, replace=False)
    # print(idx_1, idx_2)
    # Sort indices in ascendin order
    if idx_1 > idx_2:
        idx_1, idx_2 = idx_2, idx_1
    # idx_1, idx_2 = tuple(set([idx_1, idx_2]))
    # print(idx_1, idx_2)
    # Take portion of solution
    solution_slice = solution[idx_1:idx_2]
    # print(solution_slice)
    solution_remaining = solution[:idx_1] + solution[idx_2:len(solution)]
    # print(solution_remaining)
    idx_insertion = np.random.choice(range(len(solution_remaining)+1),1)
    idx_insertion = idx_insertion[0]
    # print(idx_insertion)
    # Perform crossover
    solution = solution_remaining[:idx_insertion] + solution_slice + solution_remaining[idx_insertion:]
    # print(solution)
    return tuple(solution)

def multi_point_mutation(solution, num_mutations_max=3):
    """
        Mutates a solution by switching the position of 2 genes.

        Arguments
        ---------
        solution (tuple): tuple containing a solution, i.e. an ordered list of destinations
        num_mutations_max (int): maximum number of single-point nutations

        Returns
        -------
        tuple: tuple containing a solution, i.e. an ordered list of destinations. Different from the input
    """
    solution = list(solution)
    num_mutations = np.random.randint(1, num_mutations_max)
    indices = range(len(solution))

    for i in range(num_mutations):
        idx_1, idx_2 = np.random.choice(indices, 2, replace=False)
        solution[idx_1], solution[idx_2] = solution[idx_2], solution[idx_1]
    return tuple(solution)

def run_genetic_algorithm(landmarks, landmarks_distances, landmarks_durations, 
                          population_size=200, 
                          max_generations=1000, 
                          evaluation_kind='distance'):
    """
    """

    # Define the size of the best solutions subset. In this case 20% of the total population size is used
    best_solutions_size = int(population_size *.20)

    # Generate the initial random population, containing population_size solutions
    population = generate_random_population(population_size, landmarks)

    # Iterate through generations
    for gen in range(max_generations):
        
        # Evaluate solutions' fitness
        population_fitness = {}

        for solution in population:
            if solution not in population_fitness.keys():
                population_fitness[solution] = evaluate_solution(solution, landmarks_distances, landmarks_durations, kind=evaluation_kind)
            else:
                continue
            
        offspring_population = []
        # Select the best solutions in the population and use them to produce the offspring solutions
        for rank, solution in enumerate(sorted(population_fitness, key=population_fitness.get)[:best_solutions_size]):

            if gen % (int(max_generations*.10)) == 0 and rank == 0:
                print("The best solution of generation {} is:".format(gen))
                print(solution)
                print("With a fitness of {}".format(population_fitness[solution]))
            
            # Copy the best solutions into the offspring population
            offspring_population.append(solution)
            # if len(offspring_population[-1]) != 8:
            #         print("wrong size")
            #         print(offspring_population[-1])

            # For each best solution, perform 7 crossover mutations
            for mut_idx in range(7):
                offspring_population.append(crossover_mutation(solution))
                # if len(offspring_population[-1]) != 8:
                #     print("wrong size")
                #     print(offspring_population[-1])

            # For each best solution, perform 2 multi-point mutations
            for mut_idx in range(2):
                offspring_population.append(multi_point_mutation(solution, 3))
                # if len(offspring_population[-1]) != 8:
                #     print("wrong size")
                #     print(offspring_population[-1])
        population = offspring_population
        # print("End gen")
    return(population)

In [46]:
idx_1, idx_2 = 8, 1
print(idx_1, idx_2)
# Sort indices in ascendin order
idx_1, idx_2 = tuple(set([idx_1, idx_2]))
print(idx_1, idx_2)

8 1
8 1


In [47]:
final_population = run_genetic_algorithm(landmarks_test, landmarks_distances, landmarks_durations, 
                                        population_size=100, 
                                        max_generations=1000, 
                                        evaluation_kind='distance')

The best solution of generation 0 is:
('Fortezza, Italy', 'Bolzano, Italy', 'Zagabria, Croatia', 'Lubiana, Slovenia', 'Passau, Germany', 'Budapest, Hungary', 'Cracovia, Poland', 'Praga, Czech Republic')
With a fitness of 2612503.0
The best solution of generation 100 is:
('Praga, Czech Republic', 'Passau, Germany', 'Bolzano, Italy', 'Fortezza, Italy', 'Lubiana, Slovenia', 'Zagabria, Croatia', 'Budapest, Hungary', 'Cracovia, Poland')
With a fitness of 1854005.0
The best solution of generation 200 is:
('Praga, Czech Republic', 'Passau, Germany', 'Bolzano, Italy', 'Fortezza, Italy', 'Lubiana, Slovenia', 'Zagabria, Croatia', 'Budapest, Hungary', 'Cracovia, Poland')
With a fitness of 1854005.0
The best solution of generation 300 is:
('Praga, Czech Republic', 'Passau, Germany', 'Bolzano, Italy', 'Fortezza, Italy', 'Lubiana, Slovenia', 'Zagabria, Croatia', 'Budapest, Hungary', 'Cracovia, Poland')
With a fitness of 1854005.0
The best solution of generation 400 is:
('Praga, Czech Republic', 'Pas

In [32]:
d = {'a':120, 'c':90}
'b' in d

False

In [24]:
list(range(10))[1:]

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [26]:
sorted(d)

['a', 'c']

In [35]:
sorted(d.items(), key=lambda x:x[1])

[('c', 90), ('a', 120)]

In [65]:
a = [1,2,3,4,5,6,7]

In [102]:
i1, i2 = np.random.choice(range(len(a)+1),2, replace=False)
print(i1,i2)
# Sort indices in ascendin order
tuple(set([i1, i2]))

3 5


(3, 5)

In [123]:
a1 = a[2:6]
a1

[3, 4, 5, 6]

In [124]:
a2 = a[:2]+a[6:len(a)]
a2

[1, 2, 7]

In [129]:
idx_insertion=4
a2[:idx_insertion] + a1 + a2[idx_insertion:]

[1, 2, 7, 3, 4, 5, 6]

In [97]:
list(range(len(a)))

[0, 1, 2, 3, 4, 5, 6]

In [107]:
a[:-1]

[1, 2, 3, 4, 5, 6]

In [118]:
a[:7]

[1, 2, 3, 4, 5, 6, 7]

In [122]:
i = np.random.choice(range(len(a)+1),1, replace=False)
i = i[0]
i

3